# test_quasar8_clean3_getdist_overlay

Load scenario-2 and scenario-3 `.nc` files from clean-only parent500 runs,
derive `lambda_median` per posterior sample, and plot a 2D overlay contour
for `H0` vs `lambda_median` using `getdist`.


In [ ]:
from pathlib import Path

import numpy as np
import arviz as az
import matplotlib.pyplot as plt
from getdist import MCSamples, plots

cwd = Path.cwd().resolve()
repo_root = None
if (cwd / 'slcosmo').is_dir() and (cwd / 'test').is_dir():
    repo_root = cwd
elif (cwd / 'LensedUniverse' / 'test').is_dir():
    repo_root = cwd / 'LensedUniverse'
else:
    for candidate in [cwd, *cwd.parents]:
        if (candidate / 'test').is_dir() and (candidate / 'data').is_dir():
            repo_root = candidate
            break

if repo_root is None:
    raise RuntimeError(f'Cannot locate LensedUniverse repo root from cwd={cwd}')

TEST_DIR = repo_root / 'test'
print('Repo root:', repo_root)
print('Test dir:', TEST_DIR)


In [ ]:
# Scenario-2 and Scenario-3 netcdf files
nc_s2 = TEST_DIR / 'test_quasar8_clean3_individual_mst_8pct.nc'
nc_s3 = TEST_DIR / 'test_quasar8_clean3_parent500_no_individual.nc'

idata_s2 = az.from_netcdf(nc_s2)
idata_s3 = az.from_netcdf(nc_s3)

# h0 samples
h0_s2 = np.asarray(idata_s2.posterior['h0'].values).reshape(-1)
h0_s3 = np.asarray(idata_s3.posterior['h0'].values).reshape(-1)

# lambda_true has shape (chain, draw, lens); reduce lens axis by median
lambda_true_s2 = np.asarray(idata_s2.posterior['lambda_true'].values)
lambda_true_s3 = np.asarray(idata_s3.posterior['lambda_true'].values)

lambda_median_s2 = np.median(lambda_true_s2, axis=-1).reshape(-1)
lambda_median_s3 = np.median(lambda_true_s3, axis=-1).reshape(-1)

print('Scenario-2 samples:', h0_s2.size)
print('Scenario-3 samples:', h0_s3.size)
print('S2 H0 median:', float(np.median(h0_s2)), 'lambda_median median:', float(np.median(lambda_median_s2)))
print('S3 H0 median:', float(np.median(h0_s3)), 'lambda_median median:', float(np.median(lambda_median_s3)))


In [ ]:
# Build getdist samples
s2 = MCSamples(
    samples=np.column_stack([h0_s2, lambda_median_s2]),
    names=['H0', 'lambda_median'],
    labels=[r'H_0', r'\lambda_{\rm median}'],
    label='Scenario 2: individual MST +8% (quadrature)',
)

s3 = MCSamples(
    samples=np.column_stack([h0_s3, lambda_median_s3]),
    names=['H0', 'lambda_median'],
    labels=[r'H_0', r'\lambda_{\rm median}'],
    label='Scenario 3: no individual MST, parent500 @1%',
)

g = plots.get_subplot_plotter(width_inch=6)
g.triangle_plot(
    [s2, s3],
    ['H0', 'lambda_median'],
    filled=[True, False],
    contour_colors=['#f48c06', '#2ca25f'],
    legend_labels=['Scenario 2', 'Scenario 3'],
    legend_loc='upper right',
)

png_out = TEST_DIR / 'test_quasar8_clean3_lambda_median_vs_h0_getdist.png'
pdf_out = TEST_DIR / 'test_quasar8_clean3_lambda_median_vs_h0_getdist.pdf'
plt.savefig(png_out, dpi=200, bbox_inches='tight')
plt.savefig(pdf_out, bbox_inches='tight')
plt.show()

print('Saved:', png_out)
print('Saved:', pdf_out)
